In [12]:
import pandas as pd

# Load the time_distance.csv dataset
time_distance_df = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/time_distance.csv')

# Initialize list to store the output data
output_data = []

# Iterate through each row in time_distance_df
for index, row in time_distance_df.iterrows():
    zip_code = row['zip']
    if pd.isna(row['walmarts_pot_10']):
        continue  # Skip if walmarts_pot_10 is NaN
    store_numbers = str(row['walmarts_pot_10']).split()

    for store_number in store_numbers:
        output_data.append([zip_code, store_number])

# Convert the output data to a DataFrame
output_df = pd.DataFrame(output_data, columns=['origin', 'destination'])

# Save the output DataFrame to a CSV file
output_df.to_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/final_cleaned_time_distance.csv', index=False)

In [13]:
import pandas as pd

# Load the datasets
final_cleaned_time_distance_df = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/final_cleaned_time_distance.csv')
zips_geocoded_df = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/zips_geocoded.csv')
walmart_geocoded_df = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/walmart_geocoded.csv')

# Function to concatenate latitude and longitude
def concat_lat_long(latitude, longitude):
    return f"{latitude},{longitude}"

# Add concatenated lat,long columns to zips and walmart dataframes
zips_geocoded_df['lat_long'] = zips_geocoded_df.apply(lambda x: concat_lat_long(x['latitude'], x['longitude']), axis=1)
walmart_geocoded_df['lat_long'] = walmart_geocoded_df.apply(lambda x: concat_lat_long(x['latitude'], x['longitude']), axis=1)

# Create dictionaries for fast lookups
zip_info = zips_geocoded_df.set_index('zip')['lat_long'].to_dict()
walmart_info = walmart_geocoded_df.set_index('Store #')['lat_long'].to_dict()

# Process each row to add origin and destination coordinates
final_cleaned_time_distance_df['origin_coords'] = final_cleaned_time_distance_df['origin'].map(zip_info)
final_cleaned_time_distance_df['destination_coords'] = final_cleaned_time_distance_df['destination'].map(walmart_info)

# Save the updated DataFrame
final_cleaned_time_distance_df.to_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/final_cleaned_time_distance_with_coords.csv', index=False)

In [14]:
import pandas as pd

# Load the datasets
final_df = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/final_cleaned_time_distance_with_coords.csv')
zips_geocoded_df = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/zips_geocoded.csv')

# Create a dictionary for RUCA values based on zip code
ruca_dict = zips_geocoded_df.set_index('zip')['ruca'].to_dict()

# Map RUCA values to the final dataframe based on origin
final_df['ruca'] = final_df['origin'].map(ruca_dict)

# Save the updated DataFrame
final_df.to_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/final_cleaned_time_distance_with_ruca.csv', index=False)

In [15]:
import pandas as pd
from geopy.distance import great_circle

# Load the dataset
df = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/final_cleaned_time_distance_with_ruca.csv')

# Function to calculate great circle distance
def calculate_distance(row):
    origin = tuple(map(float, row['origin_coords'].split(',')))
    destination = tuple(map(float, row['destination_coords'].split(',')))
    return great_circle(origin, destination).miles

# Apply the function to calculate distance
df['great_circle_distance'] = df.apply(calculate_distance, axis=1)

# Save the updated DataFrame
df.to_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/final_cleaned_time_distance_with_great_circle_distance.csv', index=False)

In [18]:
df = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/final_cleaned_time_distance_with_great_circle_distance.csv')

data_encoded = pd.get_dummies(df, columns=['ruca'])
data = data_encoded
data.to_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/final_cleaned_time_distance.csv', index=False)
